# QMMM workflow using GROMACS and VOTCA-XTP

## What is this tutorial about
In this tutorial, we will learn how to set and perform excited state calculation using the Votca XTP library. We will use methane as our QM region.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/doc/INSTALL.rst)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above

## Interacting with the XTP command line interface
The XTP package offers the following command line interface that the user can interact with:
* [xtp_map](https://www.votca.org/xtp/xtp_map.html)
* [xtp_parallel](https://www.votca.org/xtp/xtp_parallel.html)
* [xtp_run](https://www.votca.org/xtp/xtp_run.html)
* [xtp_tools](https://www.votca.org/xtp/xtp_tools.html)

Run the following command to view the help message of `xtp_tools`:

In [1]:
!xtp_tools -h

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_tools, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 17:32:21)
votca_csg, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 16:43:14)
votca_tools, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 16:39:10)

Runs excitation/charge transport tools



Allowed options:
  -h [ --help ]                 display this help and exit
  --verbose                     be loud and noisy
  --verbose1                    be very loud and noisy
  -v [ --verbose2 ]             be extremly loud and noisy
  -o [ --options ] arg          Tool user options.
  -t [ --nthreads ] arg (=1)    number of threads to create
  -e [ --execute ] arg        Name of Tool to run
  -l [ --list ]               Lists all available Tools
  -d [ --description ] arg    Short description of a Tools
  -c [ --cmdoptions ] arg     Modify options via command line by e.g. '-c 
 

### Note
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

## Setting the environment


Create a folder to store the input `Options` for XTP

In [2]:
!mkdir -p OPTIONFILES

Remove previous hdf5 file

In [3]:
!rm -f state.hdf5

In [4]:
from xml_editor import add_section, edit_calculator
import os

## Generate the topology from the Gromacs file

runs the mapping from MD coordinates to segments and creates an [hdf5 file](https://www.hdfgroup.org/solutions/hdf5/). You can explore the generated `state.hdf5` file with e.g. hdf5itebrowser. In Python, you can use the [h5py library](https://www.h5py.org/).

In [5]:
!xtp_map -t MD_FILES/topol.tpr -c MD_FILES/conf.gro -s system.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_map, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 17:32:21)
votca_csg, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 16:43:14)
votca_tools, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 16:39:10)

Reading file MD_FILES/topol.tpr, VERSION 5.1.1 (single precision)
Note: file tpx version 103, software tpx version 122
Creating statefile state.hdf5
Frame with id 0 was not in statefile state.hdf5 ,adding it now.
Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


##  Check the mapping

Let us first output `.pdb` files for the segments, qmmolecules and classical segments in order to check the mapping. Use `xtp_run -d mapchecker` to see all options `mapchecker` calculator takes. We use the `-c` option to change one option on the commandline.

In the [mapchecker section of the manual](https://votca.github.io/xtp/mapchecker.html) you can find a table with the `mapchecker` input variables and their corresponding defaults. Finally, the following command run the check

In [7]:
!xtp_run -e mapchecker -c map_file=system.xml -f state.hdf5

<mapchecker help="Outputs .pdb files for segments, qmmolecules and classical segments to check the mapping" injected="true" section="sec:mapchecker">
	<qm_states default="n" help="qmstates states to check">n</qm_states>
	<mp_states default="e h" help="multipole states to check">e h</mp_states>
	<map_file default="votca_map.xml" help="xml file with segment definition" injected="true">system.xml</map_file>
	<md_pdbfile default="md_segments.pdb" help="PDB file with the MD calculation">md_segments.pdb</md_pdbfile>
	<qm_pdbfile default="qm_segments.pdb" help="PDB file with the QM calculation">qm_segments.pdb</qm_pdbfile>
	<mp_pdbfile default="mp_segments.pdb" help="PDB file with the MP segments">mp_segments.pdb</mp_pdbfile>
</mapchecker>

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_run, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 17:32:21)
votca_csg, version 2022-dev gitid: b4c29aa (dirty) (compiled Ju

## Neighborlist Calculation

The following step is to determine the neighbouring pairs for exciton transport. See the [neighborlist options](https://www.votca.org/xtp/neighborlist.html) for further information. 

Finally, we can run the calculation using 4 threads

In [8]:
!xtp_run -e neighborlist -c exciton_cutoff=0.5 constant=0.6 -f state.hdf5 -t 4

<neighborlist help="Determines neighbouring pairs for exciton transport" injected="true" section="sec:neighborlist">
	<constant choices="float+" default="1.5" help="cutoff for all other types of pairs if the constant method is used" injected="true" unit="nm">0.6</constant>
	<exciton_cutoff choices="float+" default="OPTIONAL" help="cutoff for classical exciton transition charge treatment" injected="true" units="nm">0.5</exciton_cutoff>
</neighborlist>

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_run, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 17:32:21)
votca_csg, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 16:43:14)
votca_tools, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 16:39:10)

Initializing calculator
... neighborlist
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... neigh

## Read reorganization energies
In this step we will read the in site reorganization energies and store them in the `state.hdf5` file. We just need to copy the input file and execute the calculation. The side energies have to be calculated by the user beforehand and put into an xml file. We added them to `system.xml`

In [10]:
!xtp_run -e einternal -c energies_file=system.xml -f state.hdf5

<einternal help="Reads in site and reorganosation energies and writes them to the state file" injected="true" section="sec:einternal">
	<energies_file default="system.xml" help="input file with vacuum site, reorganization (charging, discharging) energies" injected="true">system.xml</energies_file>
</einternal>

========   VOTCA (http://www.votca.org)   ========

please submit bugs to https://github.com/votca/xtp/issues

xtp_run, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 17:32:21)
votca_csg, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 16:43:14)
votca_tools, version 2022-dev gitid: b4c29aa (dirty) (compiled Jul  7 2021, 16:39:10)

Initializing calculator
... einternal
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5
.... 
... einternal
 Using 1 threads
... ... Site, reorg. energies from system.xml.

... ... Read in site, reorg. energies for 1000 segments. Wrote MD topology (

## Compute site energy
In this step we will perform some *QMMM* calculations to compute the site energies. The `qmmm_mm.xml` file contains some predefined settings to perform the *MM* calculations. Let us first copy these settings into the state file. Instead of using the `-c` option we now use the `-o` option to read in options from an xml file.

In [38]:
!cp qmmm_mm.xml OPTIONFILES
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "write"

<>
	<options injected="true">
		<qmmm help="Executes qmmm calculations for individual molecules and clusters" injected="true" section="sec:qmmm">
			<print_regions_pdb choices="bool" default="true" help="print the geometry of the regions to a pdb file"/>
			<max_iterations choices="int+" default="100" help="max iterations for qmmm scf loop"/>
			<map_file default="votca_map.xml" help="xml file with segment definition" injected="true">system.xml</map_file>
			<job_file default="qmmm_jobs.xml" help="name of jobfile to which jobs are written" injected="true">qmmm_mm_jobs.xml</job_file>
			<io_states default="n e h" help="states to write jobs for and which to parse from jobfile" injected="true">n e h s1</io_states>
			<regions default="REQUIRED" help="Definitions of all regions inside qmmmm" injected="true" list="">
				<polarregion default="OPTIONAL" help="polar region with polarisation dipoles and thole damping" injected="true" link="region.xml">
					<polar help="Options for the electro

.... 
... qmmm 
... ... Writing job file qmmm_mm_jobs.xml
... ... In total 4001 jobs
Changes have not been written to state file.


The previous command generates a `qmmm_mm_jobs.xml` containing 4000 *MM* jobs to compute, if you examine that file, it should look something like

```xml
<jobs>
  <job>
    <id>0</id>
    <tag>Methane_0:n</tag>
    <input>
      <site_energies>0:n</site_energies>
      <regions>
        <region>
          <id>0</id>
          <segments>0:n</segments>
        </region>
      </regions>
    </input>
    <status>AVAILABLE</status>
  </job>    
...
```

Let us run just the first 4 jobs by settings all jobs `status` to `COMPLETE` except for the first four. This can be easily done with [sed](https://www.gnu.org/software/sed/manual/sed.html) as follows,

In [39]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml

Now we can run the jobs and save the results in the state file

In [37]:
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -x 2 -j "run"
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm_mm.xml -f state.hdf5 -j "read"

<>
	<options injected="true">
		<qmmm help="Executes qmmm calculations for individual molecules and clusters" injected="true" section="sec:qmmm">
			<print_regions_pdb choices="bool" default="true" help="print the geometry of the regions to a pdb file"/>
			<max_iterations choices="int+" default="100" help="max iterations for qmmm scf loop"/>
			<map_file default="votca_map.xml" help="xml file with segment definition" injected="true">system.xml</map_file>
			<job_file default="qmmm_jobs.xml" help="name of jobfile to which jobs are written" injected="true">qmmm_mm_jobs.xml</job_file>
			<io_states default="n e h" help="states to write jobs for and which to parse from jobfile" injected="true">n e h s1</io_states>
			<regions default="REQUIRED" help="Definitions of all regions inside qmmmm" injected="true" list="">
				<polarregion default="OPTIONAL" help="polar region with polarisation dipoles and thole damping" injected="true" link="region.xml">
					<polar help="Options for the electro

.... 
... qmmm 
MST ERR Job file = 'qmmm_mm_jobs.xml', cache size =  8
MST ERR Initialize jobs from qmmm_mm_jobs.xml
MST ERR Registered 4000 jobs.
T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack
T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ...  Regions created
T00 ERR ... Id: 0 type: polarregion size: 17 charge[e]= -9.29812e-16
T00 ERR ... Id: 1 type: staticregion size: 254 charge[e]= -1.4086e-14
T00 ERR ... 2021-7-9 21:39:48 Writing jobtopology to MMMM/frame_0/job_0_Methane_0:n/regions.pdb
T00 ERR ... 2021-7-9 21:39:48 Only 1 scf region is used. The remaining regions are static. So no inter regions scf is required. 
T00 ERR ... 2021-7-9 21:39:48 --Inter Region SCF Iteration 1 of 1
T00 ERR ... 2021-7-9 21:39:48 Evaluating polarregion 0
T00 ERR ... 2021-7-9 21:39:48 Evaluating interaction between polarregion 0 and staticregion 1
T00 ERR ... 2021-7-9 21:39:48 Starting Solving for classical polarization with 255 degrees of freedom.
T00 ERR ... 2021-7-9 21:39:48 CG: #it

.... 
... qmmm 
Found 1 states of type e
Found 1 states of type h
Found 1 states of type s
Found 1 states of type n
3996 incomplete jobs found.

Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


## Site energy and pair energy analysis
In this step we generate an histogram and compute the correlation function of site energies and pair energy differences.

In [ ]:

!xtp_run -e eanalyze -o OPTIONFILES/eanalyze.xml -f state.hdf5

You should now see a set of files prefixed with `eanalyze` containing the histrogram and correlation functions.

In [ ]:
!ls eanalyze*

## QM energy calculation
Our next task is to perform the qm calculations for each segment that we have stored in the hdf5 file. The calculations take place in 3 stages: write the jobs to a file, perform the computation and finally save the results to the state file. We will first copy the input into our local folder

In [ ]:
!cp "$VOTCASHARE/xtp/xml/eqm.xml" OPTIONFILES/

Now we set the GWBSE mode to `G0W0`,  the `ranges` to `full` and the `basisset` and `auxbasisset` to `3-21G` and `aux-def2-svp`. For more information, check the [eqm calculator options](https://votca.github.io/xtp/eqm.html).

In [ ]:
edit_calculator("eqm", "gwbse_options.gwbse.ranges", "full")
edit_calculator("eqm", "gwbse_options.gwbse.mode", "G0W0")
edit_calculator("eqm", "gwbse_options.gwbse.qp_solver", "fixedpoint")
edit_calculator("eqm", "gwbse_options.gwbse.qp_grid_steps", "100")
edit_calculator("eqm", "gwbse_options.gwbse.qp_grid_spacing", "0.01")
edit_calculator("eqm", "gwbse_options.gwbse.exctotal", "10")
edit_calculator("eqm", "map_file", "system.xml")
edit_calculator("eqm", "basisset", "3-21G")
edit_calculator("eqm", "auxbasisset", "aux-def2-svp")

For the sake of computational time let just compute the `gw` approximation and the `singlet`. You can also request the `triplet` or `all`,  see the [gwbse sectionfor the eqm calculator](https://www.votca.org/xtp/eqm.html).

In [ ]:
edit_calculator("eqm", "gwbse_options.gwbse.tasks", "gw,singlets")

First we will write the job in a file and enable only the first 2,

In [ ]:
!xtp_parallel -e eqm -o OPTIONFILES/eqm.xml -f state.hdf5 -s 0 -j "write"
!sed -i "s/AVAILABLE/COMPLETE/g" eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs


Now, let run these 2 jobs

In [ ]:
!xtp_parallel -e eqm -o OPTIONFILES/eqm.xml -f state.hdf5 -x 2 -s 0 -j run -q 1

## QM calculation for pairs
In the following step we will run QM calculations for each pair in the hdf5 file. As the calculations on the previous step, we will first write the jobs in a file, then run them and finally store the results in the state file. First, we need to copy the input to our local folder

In [ ]:
!cp "$VOTCASHARE/xtp/xml/iqm.xml" OPTIONFILES/

As in the previous section, we set the GWBSE mode to `G0W0`, the `ranges` to `full` and the `basisset` and `auxbasisset` to `3-21G` and `aux-def2-svp`. But we compute only the `gw` approximation. We are also going to compute a single excited state. For more information, check the [iqm calculator options](https://www.votca.org/xtp/iqm.html). We also want to compute the `singlet` couplings. 

In [ ]:
edit_calculator("iqm", "states", "1")
edit_calculator("iqm", "map_file", "system.xml")
edit_calculator("iqm", "gwbse_options.gwbse.ranges", "full")
edit_calculator("iqm", "gwbse_options.gwbse.tasks", "gw")
edit_calculator("iqm", "gwbse_options.gwbse.qp_solver", "fixedpoint")
edit_calculator("iqm", "gwbse_options.gwbse.qp_grid_steps", "100")
edit_calculator("iqm", "gwbse_options.gwbse.qp_grid_spacing", "0.01")
edit_calculator("iqm", "gwbse_options.gwbse.exctotal", "10")
edit_calculator("iqm", "bsecoupling.spin", "singlet")
edit_calculator("iqm", "basisset", "3-21G")
edit_calculator("iqm", "auxbasisset", "aux-def2-svp")

Before running the calculations, we need to specify in the `iqm` input which states to read into the jobfile for each segment type. The following *XML* snippet needs to be added

In [ ]:
readjobfile = """
<readjobfile help="which states to read into the jobfile for each segment type">
     <singlet>Methane:s1</singlet>
     <triplet>Methane:t1</triplet>
     <electron>Methane:e1</electron>
     <hole>Methane:h1</hole>
</readjobfile>
"""

The following code add the `readjobfile` section,

In [ ]:
add_section("iqm", readjobfile)

Now, let's write the jobs to the file

In [ ]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -s 0 -j "write"

From the jobs that we just write down, let's make available only the first job

In [ ]:
!sed -i "s/AVAILABLE/COMPLETE/g" iqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' iqm.jobs

Now we can run and store the jobs results

In [ ]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -x 2 -s 0 -j run -q 1

Finally, we read the results into the state

In [ ]:
!xtp_parallel -e iqm -o OPTIONFILES/iqm.xml -f state.hdf5 -j "read"

## Coupling
We can now compute the classical coupling of transition in the aformentioned three stages,

In [ ]:
!cp "${VOTCASHARE}/xtp/xml/iexcitoncl.xml" OPTIONFILES

We need to change in the `iexcitoncl` input the name `map_file` option and add the state. check all the available of the [iexcitoncl calculator](https://www.votca.org/xtp/iexcitoncl.html).

In [ ]:
edit_calculator("iexcitoncl", "map_file", "system.xml")

In [ ]:
state = "<states>Methane:n2s1</states>"
add_section("iexcitoncl", state)

In [ ]:
!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5 -j "write"

In [ ]:
!ls


Now we can run and save the jobs. For demo purposes we will run only the first job

In [ ]:
!sed -i "s/AVAILABLE/COMPLETE/g" exciton.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' exciton.jobs
!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5

In [ ]:
#!xtp_parallel -e iexcitoncl -o OPTIONFILES/iexcitoncl.xml -f state.hdf5 -j "read"

## Coupling analysis
Using the coupling computed in the previous steps, we will generate an histogram for the squared couplings in logarithmic scale,

In [ ]:
!cp "${VOTCASHARE}/xtp/xml/ianalyze.xml" OPTIONFILES
edit_calculator("ianalyze", "states", "e,h,s")
!xtp_run -e ianalyze -o OPTIONFILES/ianalyze.xml -f state.hdf5

## QMMM calculations

In [ ]:
!cp qmmm.xml OPTIONFILES/
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j "write"

Lets run just the first job

In [ ]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_jobs.xml                                                                                                                                                            
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_jobs.xml   
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_jobs.xml  
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -x 2 -f state.hdf5 -j run

Finally, save the results

In [ ]:
!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j "read"